In [6]:
from turtle import title
import scrapy
import json
import time
import json
from scrapy.crawler import CrawlerProcess
#import cfscrape
#from fake_useragent import UserAgent
from turtle import title
import scrapy
import json
import time
import json
#import cfscrape
#from fake_useragent import UserAgent
from scrapy.crawler import CrawlerProcess
from urllib.parse import urlencode
from urllib.parse import urljoin
from scrapy import signals
from pydispatch import dispatcher
import pandas as pd

df = pd.read_csv('C:/Users/Asus/OneDrive/Desktop/crawler/postscrape/postscrape/spiders/Book1.csv')

item = df['Name'].tolist()
starturl =[]

for w1 in item:                                                     
   starturl.append(f'https://www.amazon.in/s?k={w1}')


# w1 = input("Enter your item: ")
# starturl.append(f'https://www.amazon.in/s?k={w1}')   //  Comment line 20-22 and comment out line 24-25 to get input realtime rather than uploading csv file

# API = '17a5880c2d8754577f3991c405ad5b5a'

# def get_url(url):
#     payload = {'api_key': API, 'url': url}
#     proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)    // comment line 46-47 and comment out 29-32 and 44-45 if serval request has to made as it will avoid blockage of spider 
#     return proxy_url

class QuotesSpider(scrapy.Spider):
    name = "amazon"
    # *** Change this url for your prefered search from hemnet.se ***
    start_urls = starturl
    globalIndex = 0
    results = {}
    custom_settings = {
        'FEEDS': {
            'quotes.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    

    def start_requests(self):
        for url in self.start_urls:
            # yield scrapy.Request(url=get_url(url), dont_filter=True, meta={'start_url': url},callback=self.parse,
            # method="GET")
            yield scrapy.Request(url, dont_filter=True, meta={'start_url': url},callback=self.parse,
            method="GET")
    
    def parse(self, response):
           
            
            for ad in response.css("div.a-spacing-base "):
             
                title = ad.css("div.s-title-instructions-style > div.a-row > h5.s-line-clamp-1 > span.a-size-base-plus::text").get()
                
                prices = ad.css("div.s-price-instructions-style > div.a-row > a.s-underline-link-text > span.a-price > span.a-offscreen::text").get()
                if prices != None:
                    prices = prices.replace('\u20b9', '')

                original_price = ad.css("div.s-price-instructions-style > div.a-row > a.s-underline-link-text > span.a-text-price > span.a-offscreen::text").get()
                if original_price != None:
                    original_price = original_price.replace('\u20b9', '')

                description = ad.css("div.s-title-instructions-style > h2.a-size-mini > a.a-link-normal > span.a-size-base-plus::text").get()

                review = ad.css("div.a-spacing-top-micro > div.a-row  > span::attr(aria-label)").get()

                No_of_review = ad.css("div.a-spacing-top-micro > div.a-row  > span > a.a-link-normal > span.a-size-base::text").get()

                id = ad.css("div.s-title-instructions-style > h2.a-size-mini > a.a-link-normal::attr(href)").get()

                
                start = id.find('dp/') +3
                end = id.find('/ref')
                id = id[start:end]
                

                if len(id) == 10:
                    sponsered = "False"
                else :
                    sponsered = "True"

                if len(id) == 10:
                    id = id
                else :
                    starts = id.find('dp%2') + 4
                    ends = id.find('%2Fref') 
                    id = id[starts:ends]

                prime_tag = ad.css("div.a-spacing-top-micro > div.a-row  > div.s-align-children-center > span.s-image-logo-view > span.s-prime > i::attr(aria-label)").get()
                Deal_of_day = ad.css("div.a-spacing-top-micro > div.a-row > a.a-link-normal > span::attr(id)").get()
                best_seller = ad.css("div.a-spacing-none > div.a-link-normal  >  span.rush-component > div.a-row > span::attr(id)").get()

                
                
                start_url = response.meta['start_url']
                start1 = start_url.find('s?k=') + 4
                category = start_url[start1:]
                
                yield {
                'category' : category,
                'Deal_of_day' : Deal_of_day,
                'best_seller' : best_seller,   
                'title': title,
                'description': description,
                'price': prices,
                'original-price' : original_price,
                'review' : review ,
                'No_of_review' : No_of_review,  
                'id' : id,
                'sponsered' : sponsered,
                'prime_tag' : prime_tag,
                'start_url': response.meta['start_url'] ,
                }    
            
         
            

    

In [9]:
process = CrawlerProcess()
import sys   
if "twisted.internet.reactor" in sys.modules:
    del sys.modules["twisted.internet.reactor"]
process.crawl(QuotesSpider)
process.start()

2022-05-17 20:27:31 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-05-17 20:27:31 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 3.0.3 3 May 2022), cryptography 37.0.2, Platform Windows-10-10.0.19043-SP0
2022-05-17 20:27:31 [scrapy.crawler] INFO: Overridden settings:
{}
2022-05-17 20:27:31 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-05-17 20:27:31 [scrapy.extensions.telnet] INFO: Telnet Password: 03b776ac9c0d4dcd
2022-05-17 20:27:31 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-05-17 20:27:31 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scra

2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/s?k=pen>
{'category': 'pen', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Pentonic LINC Ball Point Pen - Blister Pack of 10 (Blue)', 'price': '100', 'original-price': None, 'review': '4.3 out of 5 stars', 'No_of_review': '8,390', 'id': 'LH193', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=pen'}
2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/s?k=pen>
{'category': 'pen', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Cello Signature Carbon Ball Pen | Blue Pack of 1 | Premium Ball Pen for Smooth Writing | Gifting Pens | Premium Pens | Cello Signature Pens', 'price': '133', 'original-price': '200', 'review': '4.3 out of 5 stars', 'No_of_review': '8,504', 'id': 'GQQCC', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=pen'}
2022-05-17 20:27:33 

2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/s?k=pen>
{'category': 'pen', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Reynolds Ball Pen I Lightweight Ball Pen With Comfortable Grip for Extra Smooth Writing I School and Office Stationery | REYNOLDS JETTER CLASSIC, 4 PENS, BLUE', 'price': '110', 'original-price': None, 'review': '3.8 out of 5 stars', 'No_of_review': '9', 'id': 'TH3Y6', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=pen'}
2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/s?k=pen>
{'category': 'pen', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Reynolds Ball Pen I Lightweight Ball Pen With Comfortable Grip for Extra Smooth Writing I School and Office Stationery | BRITE BP 25 CT JAR - BLUE', 'price': '173', 'original-price': '225', 'review': '3.7 out of 5 stars', 'No_of_review': '35', 'id': 'K7LRH', 'spo

2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/s?k=pen>
{'category': 'pen', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Hauser Billi Dx Ball pen Pack of 50, Multicolor, (62805)', 'price': '335', 'original-price': '350', 'review': '3.9 out of 5 stars', 'No_of_review': '25', 'id': 'K7FT9', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=pen'}
2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/s?k=pen>
{'category': 'pen', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Reynolds Ball Pen I Lightweight Ball Pen With Comfortable Grip for Extra Smooth Writing I School and Office Stationery | VISTA RT BP 25 PENS JAR - 20 BLUE & 5 BLACK', 'price': '218', 'original-price': '225', 'review': '5.0 out of 5 stars', 'No_of_review': '2', 'id': 'SB2BZ', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=pen'}
2

2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/bottle/s?k=bottle>
{'category': 'bottle', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Milton Alive 750 Stainless Steel Water Bottle, 750 ml, Black', 'price': '264', 'original-price': '353', 'review': '4.2 out of 5 stars', 'No_of_review': '4,275', 'id': '9RJNJ', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=bottle'}
2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/bottle/s?k=bottle>
{'category': 'bottle', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Amazon Brand - Solimo Plastic Water Bottle Set (6 pieces, Multicolor, Dotted pattern)', 'price': '429', 'original-price': '875', 'review': '4.0 out of 5 stars', 'No_of_review': '1,203', 'id': 'P94V2', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=bottle'}
2022-05-17 20:27:33 [scrapy.core.scra

...

2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/bottle/s?k=bottle>
{'category': 'bottle', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Pexpo Stainless Steel Wide Mouth Sports/Fridge/Refrigerator Single Wall Leakproof Water Bottle with 3X DURABLE Color Coating ( Pack of 1, Black, Capacity 1000 ml, Chromo Bottle)', 'price': '323', 'original-price': '399', 'review': '3.9 out of 5 stars', 'No_of_review': '239', 'id': '3SJC8', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=bottle'}
2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/shoe/s?k=shoe>
{'category': 'shoe', 'Deal_of_day': None, 'best_seller': None, 'title': 'Adidas', 'description': "Men's Quickflow M Running Shoe", 'price': '2,127', 'original-price': '3,999', 'review': '3.8 out of 5 stars', 'No_of_review': '34', 'id': '4P3JH', 'sponsered': 'True', 'prime_tag': 'Amazon Prime', 'start_url': 'htt

2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/shoe/s?k=shoe>
{'category': 'shoe', 'Deal_of_day': None, 'best_seller': None, 'title': 'FLITE', 'description': 'Men Shoes', 'price': '249', 'original-price': None, 'review': '3.6 out of 5 stars', 'No_of_review': '3,541', 'id': 'DRRSH', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=shoe'}
2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/shoe/s?k=shoe>
{'category': 'shoe', 'Deal_of_day': None, 'best_seller': None, 'title': 'JointlyCreating', 'description': 'Mens Casual Mesh Walking Shoes Outdoor Sport Sneakers', 'price': '6,325', 'original-price': None, 'review': '3.8 out of 5 stars', 'No_of_review': '1,225', 'id': 'H3WQK', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=shoe'}
2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/bottle/s?k=bottle>
{'category':

2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/shoe/s?k=shoe>
{'category': 'shoe', 'Deal_of_day': None, 'best_seller': None, 'title': 'Nivia', 'description': 'Super Court Badminton Shoe', 'price': '1,439', 'original-price': '1,499', 'review': '4.1 out of 5 stars', 'No_of_review': '498', 'id': 'KHJU0', 'sponsered': 'True', 'prime_tag': 'Amazon Prime', 'start_url': 'https://www.amazon.in/s?k=shoe'}
2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/bottle/s?k=bottle>
{'category': 'bottle', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Milton Super 1000 Single Wall Stainless Steel Bottle, 1000 ml, Silver,Set of 1', 'price': None, 'original-price': None, 'review': '3.9 out of 5 stars', 'No_of_review': '6,961', 'id': 'NJJ7L', 'sponsered': 'True', 'prime_tag': None, 'start_url': 'https://www.amazon.in/s?k=bottle'}
2022-05-17 20:27:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https:

2022-05-17 20:27:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/bottle/s?k=bottle>
{'category': 'bottle', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Milton Flip Lid 1000 Thermosteel 24 Hours Hot and Cold Water Bottle with Bag, 1 Litre, Silver', 'price': '964', 'original-price': '1,205', 'review': '4.3 out of 5 stars', 'No_of_review': '6,577', 'id': 'R1SHL', 'sponsered': 'True', 'prime_tag': 'Amazon Prime', 'start_url': 'https://www.amazon.in/s?k=bottle'}
2022-05-17 20:27:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/bottle/s?k=bottle>
{'category': 'bottle', 'Deal_of_day': None, 'best_seller': None, 'title': None, 'description': 'Crevizon 3 Liter Gym Water Bottle One Click Flip Lid with Straw & Time Marker Reusable,Ideal for Adults,Gym,Outdoor Sport (Color Purple)', 'price': '599', 'original-price': '1,999', 'review': '4.8 out of 5 stars', 'No_of_review': '5', 'id': 'MQRLN', 'sponsered': 'True', 'prime_tag'

2022-05-17 20:27:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/shoe/s?k=shoe>
{'category': 'shoe', 'Deal_of_day': None, 'best_seller': None, 'title': 'Tiger', 'description': "Tiger Men's Leather Leopard High Ankle with Steel Toe Safety Shoes (Black, Size 9 UK )", 'price': '989', 'original-price': '994', 'review': '4.0 out of 5 stars', 'No_of_review': '3,114', 'id': 'G8D62', 'sponsered': 'True', 'prime_tag': 'Amazon Prime', 'start_url': 'https://www.amazon.in/s?k=shoe'}
2022-05-17 20:27:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.in/shoe/s?k=shoe>
{'category': 'shoe', 'Deal_of_day': None, 'best_seller': None, 'title': 'Adidas', 'description': "Men's Alliver M Leather Running Shoe", 'price': '1,544', 'original-price': '2,499', 'review': '4.3 out of 5 stars', 'No_of_review': '6', 'id': 'SF1BF', 'sponsered': 'True', 'prime_tag': 'Amazon Prime', 'start_url': 'https://www.amazon.in/s?k=shoe'}
2022-05-17 20:27:34 [scrapy.core.scraper] DEBUG: 